<a href="https://colab.research.google.com/github/hectormelo/Machine-Learning-Techniques/blob/main/Taller_1/Taller_1.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" />
</a>


<img src="https://bloqueneon.uniandes.edu.co/content/enforced/248650-202410_ISIS4219_01/Laboratorios.png" ><br>
# Machine Learning Techniques - ISIS4219

Primer Semestre - 2024




## Integrantes

*   Andrés Felipe Aristizábal Miranda - 202416947
*   Leandro Rodríguez Llosa - 202410038

## **Problema**

Las enfermedades cardiovasculares son la principal causa de muerte en el mundo, y se calcula que cobran 17,9 millones de vidas al año (OMS). La enfermedad de las arterias coronarias es el tipo más común de enfermedad cardíaca y se produce debido a las obstrucciones (placa) desarrolladas en el interior de las arterias coronarias (vasos sanguíneos que alimentan los músculos del corazón). Los cardiólogos utilizan diversas técnicas de imagen y mediciones invasivas de la presión arterial para examinar y controlar la gravedad de dichas obstrucciones. <br>
Los factores de riesgo conductuales más importantes de estas enfermedades son una dieta poco saludable, la inactividad física, el consumo de tabaco y el uso nocivo del alcohol. Los efectos de los factores de riesgo pueden manifestarse en las personas en forma de aumento de la presión arterial, aumento de la glucosa en sangre, aumento de los lípidos en sangre y sobrepeso y obesidad.
Identificar a las personas con mayor riesgo de sufrir enfermedades cardiovasculares y garantizar que reciban el tratamiento adecuado puede evitar muertes prematuras. Con este objetivo en mente, se quiere utilizar las técnicas de machine learning para construir un modelo que permita predecir qué pacientes pueden estar en riesgo de padecer este tipo de cardiopatía.<br><br>
Referencias. <br>



OMS (s.f.). “Cardiovascular diseases”. https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1
<br><br>
**Fuente de Datos**: https://www.kaggle.com/agsam23/coronary-artery-disease/version/3


## Instalaciones e importaciones

In [ ]:
%pip install pandas ydata-profiling scikit-learn sklearn_features matplotlib
%matplotlib inline

In [ ]:
#Data management Libraries
import pandas as pd
from ydata_profiling import ProfileReport

# Data Modeling Libraries
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, MaxAbsScaler, PolynomialFeatures, StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Data visualization Libraries
import matplotlib.pyplot as plt

#Save the model
from joblib import dump, load

# **1. Procesamiento y entendimiento de los datos.**

Explore el conjunto de datos proporcionado y prepárelos para el entrenamiento y validación. Dado que este conjunto de datos no incluye un grupo de test, es necesario realizar una división entre entrenamiento y test. Utilice la función train_test_split de sklearn.model_selection para llevar a cabo esta tarea.

¿Que particularidades presentan los datos y qué técnicas de preprocesamiento se pueden aplicar?



In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/leandro-driguez/Machine-Learning-Techniques/dev/Taller_1/Coronary_artery.csv')

data.head()

In [ ]:
ProfileReport(data)

In [ ]:
plt.bar(data['class'].unique(), data['class'].value_counts())
plt.title('Class')
plt.xlabel('Amount')
plt.ylabel('Class')
plt.show()

# **2.  Limpieza y preparación de datos**

Realizar el proceso de limpieza de datos con el fin de asegurar que los datos estén libres de errores, valores nulos, valores atípicos y formatos inconsistentes.

In [ ]:
X = data.drop(['class'], axis=1)
Y = data['class']

X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42, stratify=Y)

# **3. Modelamiento**

El proceso de modelado se refiere a la etapa en la que se construyen y entrenan modelos de machine learning para realizar predicciones o tomar decisiones basadas en datos. En este paso es importante realizar la Selección de Hiperparámetros con el fin de optimizar el modelo.

In [ ]:
pipe = Pipeline([
    ('encoder', OneHotEncoder()),
    ('normalize', MaxAbsScaler()),
    ('features_filter', SelectKBest()),
    ('poly', PolynomialFeatures()),
    ('clf', LogisticRegression(solver='liblinear', n_jobs=-1))
])

In [ ]:
param_grid = {
    'encoder': [OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
                OneHotEncoder(handle_unknown='ignore')],
    'normalize': ['passthrough', MinMaxScaler(), StandardScaler()],
    'features_filter__k': [7, 10, 'all'],
    'poly__degree': [1, 2, 3, 4],
    'clf__penalty': [None, 'l1', 'l2'],
    'clf__C': [1e-4, 1e-1, 1e2, 1e5]
}

In [ ]:
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=10, verbose=3, scoring='accuracy')

grid_search.fit(X_train, y_train.ravel())

In [ ]:
print(grid_search.best_params_)

In [ ]:
resultados = pd.DataFrame(grid_search.cv_results_)
display(resultados)

# **4. Predicción**

Una vez realizado el entrenamiento del modelo es necesario realizar una predicción con los datos de prueba, que servira como insumo para la evaluación y/o validación de la calidad del modelo.

In [ ]:
dump(grid_search.best_estimator_, 'my_model.joblib')

model = load('my_model.joblib')

pred = model.predict(X_test)
print('----------Reporte para el Mejor Modelo----------')
print(classification_report(y_test, pred))

In [ ]:
print(y_test.ravel())
print(pred)

In [ ]:
conf_matrix = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(conf_matrix)
disp.plot(cmap='Blues', values_format='d')
# Añade etiquetas y muestra el gráfico
plt.title('Matriz de Confusión')
plt.show()